# MPModel: A program to the detergent corona around a membrane protein

Throughout this tutorial, we will buid a coarse-grained model of a micelle around the  membrane protein Aquaporin-0 (PDB: 2B6P [1]) and use the `simSPI` package to simulate a cryo-EM experiment of the structure. For that, it is necessary to have installed the TEM-simulator [2] and the python libraries: Matplotlib Numpy, Gemmi, Scikit-learn and Scipy.


<p align="center">
    <img src="figures/2b6p_map.png" alt="Drawing" style="width: 700px;"/>
    <p style="text-align: center">
        Figure 1: _Bos taurus_ Aquaporin-0 complex
    </p>
</p>


[1] https://www.rcsb.org/structure/2B6P

[2] http://tem-simulator.sourceforge.net/

In [12]:
import numpy as np
import atomic_models
import gemmi
import detergent_belt as mpm
from simSPI import tem

ModuleNotFoundError: No module named 'simSPI'

## Creating the model
### Extracting the protein atomic coordinates from a PDB file

In [7]:
aqp = mpm.MembraneProtein("2b6p.pdb") #path to pdb file

<p align="center">
    <img src="figures/aquaporin.png" alt="Drawing" style="width: 400px;"/>
    <p style="text-align: center">
        Figure 2: Representation of the atomic coordinates of aquaporin
    </p>
</p>

### Centering and rotating the structure
<p style='text-align: justify;margin-right:10%;'>
Using Principal Component Analysis (PCA), a method for data decomposition, we can extract the main axis where our data is distributed. We can use a matrix transformation as a function to represent a modification on our dataset. In this case, the transformation is aligning the principal component of the coordinate points to the Z-axis. Therefore, we can distribute the atoms on another axis and, by doing so, rotate the protein so that its transmembrane portion is positioned properly in space.
</p>

In [8]:
aqp.rotate_protein(axis='X')

<p align="center">
    <img src="figures/aquaporin_rotated.png" alt="Drawing" style="width: 400px;"/>
    <p style="text-align: center">
        Figure 3: Aquaporin structure rotated and centered at the origin
    </p>
</p>

***
### Creating a corona around the protein

To create the Ellipsoid we will compute the points that satisfy the condition: 
$\frac{x^{2}}{a^{2}} + \frac{y^{2}}{b^{2}} + \frac{z^{2}}{c^{2}} >= 1$

Consider $O = (0,0,0)$, as the protein was centered after the coordinates were imported.

<p align="center">
    <img src="figures/ellipsoid.png" alt="Drawing" style="width: 200px;"/>
    <p style="text-align: center">
        Figure 4: The geometric parameters of an ellipsoid
    </p>
</p>

The method used in this program creates an ellipsoid by generating points iteratively in regular intervals dependent on the ray of the pseudoatom. After that, a function removes the coordinates in the most inner part of the ellipsoid that would conflict with the protein's atoms. This process' speed depends on the number of pseudo atoms, that itself depends on the parameters of the corona such as the height, determined by size of the transmembrane portion of the protein, and on the ray of the pseudo atoms filling the ellipsoid.

Let us create the hydrophobic interior ellipsoid of the corona with parameters chosen empirically for a better visualization of the exemple on Pymol.</br>

major axis &rarr; 35Å</br>
minor axis &rarr; 25Å</br>
ellipticity &rarr; 2</br>
pseudo atom type &rarr; CA</br>
pseudo atom ray &rarr; 1.5</br>

major axis &rarr; 40Å</br>
minor axis &rarr; 30Å</br>
ellipticity &rarr; 2</br>
pseudo atom type &rarr; N</br>
pseudo atom ray &rarr; 1.5

In [9]:
C = mpm.BeltCore()
C.set_belt_parameters(35, 25, 2)    #set the parameters for the core of your micelle
C.set_atomic_parameters(1.5, "CA")  #set the parameters for you hydrophilic pseudoatom
C.generate_ellipsoid()

S = mpm.BeltShell()
S.set_belt_parameters(40, 30, 2)    #set the parameters for the shell of your micelle
S.set_atomic_parameters(1.5, "N")   #set the parameters for you hydrophobic pseudoatom
S.generate_ellipsoid()

### Excluding the overlaping pseudoatoms
Now that we have our two ellipsoids we need to delete from our corona the pseudo atoms that are physically overlapping with other atoms
Observation: the next steps take a couple of minutes to run

In [10]:
C.in_hull(aqp.final_coordinates)
C.remove()
 
S.in_hull(C.coordinates_set)
S.remove()
S.in_hull(aqp.final_coordinates)      
S.remove()

### Generating the final PDB file

In [14]:
file_name = "outputs/my_model.pdb"   #name your pdb file
M = mpm.Model()
M.clean_gemmi_structure()
M.write_atomic_model(file_name, model=gemmi.Model("model"))
M.create_model(file_name,C,S,aqp)

<p align="center">
    <img src="figures/aqp_model_abc.png" alt="Drawing" style="width: 600px;"/>
    <p style="text-align: center">
        Figure 5: Model of detergent micelle and membrane protein Aquaporin (AQP0) seen as A- full model, B- micelle
hydrophobic core and C- frontal cut. The micelle hydrophobic core is represented in yellow,
the hydrophilic shell in blue and the protein in pink.
    </p>
</p>

## TEM-simulator